# REAL ESTATE DATA API

## Imports

In [ ]:
import subprocess
import warnings
from datetime import datetime

import pandas as pd
import requests

warnings.filterwarnings('ignore')


## Generating Dataset

### API Props

In [ ]:
client_secret = "secret_9d2aeb1d94d5f400c8128c5199a8d8de"
client_id = "client_2bf50cfd2349026b489480f4c464f395"

raw = subprocess.check_output("node getToken.js", shell=True)
bearer = raw.decode("utf-8").strip()

In [ ]:
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {bearer}"
}

In [ ]:
# URLs
listings = "https://api.domain.com.au/v1/listings/residential/_search"

In [ ]:
req_body = lambda pg_no: {
  "listingType":"Sale",
  "propertyTypes":[
    "House"
  ],
  "minBedrooms":2,
  "maxBedrooms":5,
  "minPrice": 350000,
  "maxPrice": 800000,
  "listingAttributes": [
    "HasPhotos"
  ], 
  "locations":[
    {
      "state":"VIC",
      "region":"Melbourne Region",
      "area":"",
      "suburb":"",
      "postCode":""
    }
  ],
  "excludePriceWithheld": True,
  "excludeDepositTaken": True,
  "pageSize": 200,
  "pageNumber": pg_no
}

### Health Check

In [ ]:
resp = requests.post(url=listings, json=req_body(1), headers=headers)
resp_json = resp.json()
print(resp)

### Dataframe Writer

In [ ]:
def update_df(df, resp_json):
  for listing in resp_json:
    if "listing" in listing:
      house = listing['listing']
      full_dict = {}
      full_dict.update({k: v for k,v in house.items() if k in ['listingType', 'id', 'dateListed']})
      full_dict.update(house['priceDetails'])
      full_dict.update({k: v for k,v in house['propertyDetails'].items() if k in ['propertyType', 'bathrooms', 'bedrooms', 'carspaces', 'unitNumber', 'streetNumber', 'street', 'area', 'region', 'suburb', 'postcode', 'displayableAddress', 'landArea']})
      df = df.append(full_dict, ignore_index=True)
  return df

### Main

In [ ]:
if __name__ == "__main__":
    FULL_DF = pd.DataFrame()

    for i in range(1, 10):
        resp = requests.post(url=listings, json=req_body(i), headers=headers)
        resp_json = resp.json()
        
        # process data and update df
        FULL_DF = update_df(FULL_DF, resp_json)

        if len(resp_json) < 200:
            break

In [ ]:
FULL_DF

### Save to CSV

In [ ]:
now = datetime.now().strftime("%Y-%m-%d")
FULL_DF.to_csv(f"houses-{now}.csv")

## Other Fields # TODO

In [ ]:
# rent history/appreciation
# price hsitoer/appreciation
# rent frequency
# Distance to city
# Distance to schools
# Median House Price
# Listing URL, https://developer.domain.com.au/docs/latest/apis/pkg_agents_listings/references/listings_get, "seoUrl"

## Data Visualisation #TODO

In [ ]:
# Choose CSV file (default latest)
# Plotly graphs, scatterplots - ensure popup thingos include address
# Geo plot?